# BN

## 技巧
- 前几种优化之后78
- 数据增强后已经到了82
- bn之后 到了86

## data prepocessing

In [1]:
import tensorflow as tf
import os
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

CIFAR_DIR = './cifar-10-batches-py'
print(os.listdir(CIFAR_DIR))

['batches.meta', 'data_batch_1', 'data_batch_2', 'data_batch_3', 'data_batch_4', 'data_batch_5', 'readme.html', 'test_batch']


In [10]:
def load_data(filename):
    """read data from data file."""
    with open(filename, 'rb') as f:
        data = pickle.load(f, encoding='bytes')
        return data[b'data'], data[b'labels']

# tensorflow.Dataset.
class CifarData:
    def __init__(self, filenames, need_shuffle):
        all_data = []
        all_labels = []
        for filename in filenames:
            data, labels = load_data(filename)
            all_data.append(data)
            all_labels.append(labels)
        # hstack vstack 用于合并二维数组变成矩阵形式
        self._data = np.vstack(all_data)
        self._data = self._data / 127.5 - 1
        # 小心
        self._labels = np.hstack(all_labels)
        print(self._data.shape)
        print(self._labels.shape)
        
        self._num_examples = self._data.shape[0]
        self._need_shuffle = need_shuffle
        self._indicator = 0
        if self._need_shuffle:
            self._shuffle_data()
            
    def _shuffle_data(self):
        # [0,1,2,3,4,5] -> [5,3,2,4,0,1]
        p = np.random.permutation(self._num_examples)
        self._data = self._data[p]
        self._labels = self._labels[p]
    
    def next_batch(self, batch_size):
        """return batch_size examples as a batch."""
        end_indicator = self._indicator + batch_size
        if end_indicator > self._num_examples:
            if self._need_shuffle:
                self._shuffle_data()
                self._indicator = 0
                end_indicator = batch_size
            else:
                raise Exception("have no more examples")
        if end_indicator > self._num_examples:
            raise Exception("batch size is larger than all examples")
        batch_data = self._data[self._indicator: end_indicator]
        batch_labels = self._labels[self._indicator: end_indicator]
        self._indicator = end_indicator
        return batch_data, batch_labels

train_filenames = [os.path.join(CIFAR_DIR, 'data_batch_%d' % i) for i in range(1, 6)]
test_filenames = [os.path.join(CIFAR_DIR, 'test_batch')]

train_data = CifarData(train_filenames, True)
test_data = CifarData(test_filenames, False)

(50000, 3072)
(50000,)
(10000, 3072)
(10000,)


## model


In [8]:
tf.reset_default_graph() # 利用这个可清空defualt graph以及nodes

x = tf.placeholder(tf.float32, [None, 3072])
y = tf.placeholder(tf.int64, [None])
# [None], eg: [0,5,6,3]
x_image = tf.reshape(x, [-1, 3, 32, 32])
# 32*32
x_image = tf.transpose(x_image, perm=[0, 2, 3, 1])
## 进一步封装
'''
def conv_wrapper(input_x,name,
                 output_channel,
                 strides=(3,3),padding='same',
                 kernel_initializer=None,
                ):
    """
    进一步封装卷积层
    """
    return tf.layers.conv2d(input_x
                             output_channel, # output channel number
                             strides = strides, # kernel size
                             padding = padding,
                             activation = activation,
                             kernel_initializer = kernel_initializer
                             name = name)
'''

def pooling_wrapper(input_x,name):
    """
    进一步封装pooling层
    """
    return tf.layers.max_pooling2d(input_x,
                                  (2, 2), # kernel size
                                  (2, 2), # stride
                                  name = name)
is_training = tf.placeholder(tf.bool,[]) # 没有dim bool value

def conv_wrapper(input_x,name,
                 output_channel,
                 activation,
                 is_training,
                 kernel_size=(3,3),
                 padding='same',
                 kernel_initializer=None,
                ):
    """
    进一步封装卷积层 但是这里有了bn操作在activation之前
    """
    conv2d = tf.layers.conv2d(input_x,
                             output_channel, # output channel number
                             kernel_size = kernel_size, # kernel size
                             padding = padding,
                             activation = None,
                             kernel_initializer = kernel_initializer,
                             name = name)
    bn = tf.layers.batch_normalization(conv2d,training=is_training)
    # activation 本身就是一个激活函数 可以指定自己的函数
    # 这正是灵活性 所以也可以直接作用在输出上
    return activation(bn)

def vggnet(x_image,activation,kernel_initializer=None):
    # conv1: 神经元图， feature_map, 输出图像
    conv1_1 = conv_wrapper(x_image,'conv1_1',32,activation,is_training)
    conv1_2 = conv_wrapper(conv1_1,'conv1_2',32,activation,is_training)
    conv1_3 = conv_wrapper(conv1_2,'conv1_3',32,activation,is_training)
    pooling1 = pooling_wrapper(conv1_3,'pool1')
    
    conv2_1 = conv_wrapper(pooling1,'conv2_1',64,activation,is_training)
    conv2_2 = conv_wrapper(conv2_1,'conv2_2',64,activation,is_training)
    conv2_3 = conv_wrapper(conv2_2,'conv2_3',64,activation,is_training)
    pooling2 = pooling_wrapper(conv2_3,'pool2')
    
    conv3_1 = conv_wrapper(pooling2,'conv3_1',128,activation,is_training)
    conv3_2 = conv_wrapper(conv3_1,'conv3_2',128,activation,is_training)
    conv3_3 = conv_wrapper(conv3_2,'conv3_3',128,activation,is_training)
    pooling3 = pooling_wrapper(conv3_3,'pool3')

    # [None, 4 * 4 * 32]
    flatten = tf.layers.flatten(pooling3)
    return flatten

flatten = vggnet(x_image,tf.nn.relu,None)
# flatten = vggnet(x_image,tf.nn.relu,tf.truncated_normal_initializer)
# flatten = vggnet(x_image,tf.nn.relu,tf.keras.initializers.he_uniform)
# flatten = vggnet(x_image,tf.nn.selu,None)


y_ = tf.layers.dense(flatten, 10)

loss = tf.losses.sparse_softmax_cross_entropy(labels=y, logits=y_)
# y_ -> sofmax
# y -> one_hot
# loss = ylogy_

# [1,0,1,1,1,0,0,0]
correct_prediction = tf.equal(tf.argmax(y_, 1), y)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float64))

with tf.name_scope('train_op'):
    # 自适应的小心初始rate容易飞 但是分自适应的初始学习率也要小一点
    train_op = tf.train.AdamOptimizer(learning_rate = -3,
                                      beta1 = 0.9,
                                      beta2 = 0.99).minimize(loss)
#     train_op = tf.train.MomentumOptimizer(learning_rate = 1e-4,
#                                           momentum=0.9).minimize(loss)
#     train_op = tf.train.RMSPropOptimizer(learning_rate=1e-3,
#                                          decay=0.9).minimize(loss)


## saver路径指定

In [11]:
run_dir = './run'
model_dir = os.path.join(run_dir,'model_save')
if not os.path.exists(model_dir):
    os.mkdir(model_dir)
'''
saver 初始化时候需要已经有需要保存的变量
在调用save的时候只需要传入sess作为当前状态即可
相当于先声明saver需要保存的量 就是saver功能
之后具体执行saver
可以指定每隔多少分钟/多少次保存一次
注意默认只会保存后五次 保存为ckp checkpoint文件
index data meta
saver 也会在graph中占据名字 所以需要小心命名问题
'''
saver = tf.train.Saver()

def get_model_path(model_name):
    model_path = os.path.join(model_dir,model_name)
    return model_path



## train+saver操作 

In [13]:
# cfg = tf.ConfigProto(allow_soft_placement=True )
# cfg.gpu_options.allow_growth = True
init = tf.global_variables_initializer()

batch_size = 20
train_steps = 1000
test_steps = 100
# save control
output_model_every_steps = 100

# train 10k: 71.35%
with tf.Session() as sess:
    sess.run(init)
    # reload
    # zrestore需要的是 model而不是具体文件名
    model_path = get_model_path('ckp-1000')
    if os.path.exists(model_path + '.index'):
        saver.restore(sess,model_path)
        print('model restore from %s successfully' % model_path)
    else:
        print('model restore from %s failedly' % model_path)
        
        
    for i in range(train_steps):
        batch_data, batch_labels = train_data.next_batch(batch_size)
        loss_val, acc_val, _ = sess.run(
            [loss, accuracy, train_op],
            feed_dict={
                x: batch_data,
                y: batch_labels,
                is_training: True
            })
        if (i+1) % 50 == 0:
            print('[Train] Step: %d, loss: %4.5f, acc: %4.5f' 
                  % (i+1, loss_val, acc_val))
        if (i+1) % 200 == 0:
            test_data = CifarData(test_filenames, False)
            all_test_acc_val = []
            for j in range(test_steps):
                test_batch_data, test_batch_labels \
                    = test_data.next_batch(batch_size)
                test_acc_val = sess.run(
                    [accuracy],
                    feed_dict = {
                        x: test_batch_data, 
                        y: test_batch_labels,
                        is_training: False
                    })
                all_test_acc_val.append(test_acc_val)
            test_acc = np.mean(all_test_acc_val)
            print('[Test ] Step: %d, acc: %4.5f'
                  % (i+1, test_acc))
        if (i+1) % output_model_every_steps == 0:
            saver.save(sess,
                      os.path.join(model_dir,'ckp-%04d' % (i+1)))
            # %04d 指定4宽度 不足用0补充
            print('model save to ckp-%04d' % (i+1))

model restore from ./run\model_save\ckp-1000 failedly


UnknownError: Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[node conv1_1/Conv2D (defined at <ipython-input-8-0fc43009a7e2>:55)  = Conv2D[T=DT_FLOAT, data_format="NCHW", dilations=[1, 1, 1, 1], padding="SAME", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true, _device="/job:localhost/replica:0/task:0/device:GPU:0"](Reshape, conv1_1/kernel/read)]]

Caused by op 'conv1_1/Conv2D', defined at:
  File "d:\anacoda_install\Lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "d:\anacoda_install\Lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "e:\environments\mytf\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "e:\environments\mytf\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "e:\environments\mytf\lib\site-packages\ipykernel\ke